# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [51]:
import pandas as pd
from numpy import random 
import requests
from pprint import pprint
from config import weather_api_key
import json

## Generate Cities List

In [52]:
rand_lat_gen = random.randint(-90,91,500)
rand_lon_gen = random.randint(-180,181,500)

city_corr = {"rand_lat": rand_lat_gen,
            "rand_lon": rand_lon_gen}

city_corr = pd.DataFrame(city_corr)
city_corr

,rand_lat,rand_lon
0,19,25
1,51,-53
2,10,-129
3,-76,40
4,78,-111
...,...,...
495,39,-180
496,64,140
497,86,91
498,3,60


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [70]:
city_nums = []
city_names = []
cloudss = []
countrys = []
dates = []
humiditys = []
lats = []
lngs = []
max_temps = []
wind_speeds = []

for i in city_corr.index:
    latitude = city_corr.iloc[int(i),0]
    longitude = city_corr.iloc[int(i),1]
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={weather_api_key}&cnt=1&units=imperial"
    cities = requests.get(url).json()
    
    city_num = cities['id']
    city_name = cities['name']
    clouds = cities['clouds']['all']
    country = cities['sys']['country']
    date = cities['dt']
    humidity = cities['main']['humidity']
    lat = cities['coord']['lat']
    lng = cities['coord']['lon']
    max_temp = cities['main']['temp_max']
    wind_speed = cities['wind']['speed']
    
    city_nums.append(city_num)
    city_names.append(city_name)
    cloudss.append(clouds)
    countrys.append(country)
    dates.append(date)
    humiditys.append(humidity)
    lats.append(lat)
    lngs.append(lng)
    max_temps.append(max_temp)
    wind_speeds.append(wind_speed)

    print(f"Data found: {city_num} -- {city_name}")
    
city_corr['City ID'] =  city_nums
city_corr['City'] =  city_names
city_corr['Country'] =  countrys
city_corr['Cloudiness'] =  cloudss
city_corr['Date'] =  dates
city_corr['Humidity'] =  humiditys
city_corr['Lat'] =  lats
city_corr['Lng'] =  lngs
city_corr['Max Temp'] =  max_temps
city_corr['Wind Speed'] =  wind_speeds
    
city_corr

Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 3451004 -- Rochedo
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 2988893 -- Oye-Plage
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 2462900 -- Ad Dchira
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 7306518 -- Utrik
Data found: 0 -- 
Data found: 98862 -- An Najaf
Data found: 0 -- 
Data found: 6142001 -- Schefferville
Data found: 4063619 -- Gadsden
Data found: 116401 -- Semnan Province
Data found: 0 -- 
Data found: 6098471 -- Parker Landing
Data found: 5879302 -- Ambler
Data found: 0 -- 
Data found: 2022454 -- Khingansk
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 3356264 -- Ke

Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 5767896 -- Philip
Data found: 0 -- 
Data found: 2122293 -- Ozernovskiy
Data found: 0 -- 
Data found: 535839 -- Leshukonskoye
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 395014 -- Sidīs
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 3432990 -- Labardén
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 3043895 -- Tisztviselőtelep
Data found: 0 -- 
Data found: 0 -- 
Data found: 2120591 -- Tilichiki
Data found: 1799319 -- Niuchehe
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 0 -- 
Data found: 5517580 -- Brewster
Data found: 0 -- 
Data found: 2013918 -- Ust-Maya
Data found: 0 -- 
Data found: 0 -- 
Data found: 1649150 --

ValueError: Length of values (0) does not match length of index (500)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression